# INTRODUCTION / BUSINESS PROBLEM:

* TO analysis the toronto neighbour data and based on that data, we need to find out if any person is getting job offer in that paricular location so whether that person should move to that location or not ? 

# SOURCE DATA : Wikipedia 
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
To provide the stakeholders the necessary information about data :

# DATA COLLECTION 

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood=pd.DataFrame(df[0])

In [10]:
neighborhood.head()

,0,1,2
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [11]:
neighborhood.shape

(181, 3)

In [18]:
neighborhood.columns =['Postal code', 'Borough','Neighborhood']

In [20]:
neighborhood.head()

,Postal code,Borough,Neighborhood
0,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [22]:
neighborhood.shape

(181, 3)

In [27]:
toronto_df=neighborhood.drop([neighborhood.index[0]])

In [28]:
toronto_df.head()

,Postal code,Borough,Neighborhood
1,M1A,Not assigned,NaN
2,M2A,Not assigned,NaN
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront


In [29]:
toronto_df.shape

(180, 3)

In [31]:
toronto_df.drop(toronto_df.index[toronto_df['Borough'] =="Not assigned"], inplace = True)

In [32]:
toronto_df.shape

(103, 3)

In [33]:
toronto_df.head()

,Postal code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


# METHODOLOGY:

For this report I will use a k cluster method that could help to new joiner to shift that particular location is good or not 


END OF PART 1